<a href="https://colab.research.google.com/github/bethoventigauw/MachineLearningTasks/blob/main/7thWeekTasks/XGBOOST_Google_PS_Apps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Nama   : Leonardo Caprio
#NIM    : 1103184204
#Kelas  : TK-44-G4

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

In [4]:
# Define the path to your Google Play Store Apps dataset CSV file
playstore_dataset_path = '/content/drive/MyDrive/dataset/googleplaystore.csv'

In [5]:
# Use Pandas to read the CSV file
playstore_data = pd.read_csv(playstore_dataset_path)

In [6]:
# Display the first few rows of the dataset
print(playstore_data.head())

                                                 App        Category  Rating  \
0     Photo Editor & Candy Camera & Grid & ScrapBook  ART_AND_DESIGN     4.1   
1                                Coloring book moana  ART_AND_DESIGN     3.9   
2  U Launcher Lite – FREE Live Cool Themes, Hide ...  ART_AND_DESIGN     4.7   
3                              Sketch - Draw & Paint  ART_AND_DESIGN     4.5   
4              Pixel Draw - Number Art Coloring Book  ART_AND_DESIGN     4.3   

  Reviews  Size     Installs  Type Price Content Rating  \
0     159   19M      10,000+  Free     0       Everyone   
1     967   14M     500,000+  Free     0       Everyone   
2   87510  8.7M   5,000,000+  Free     0       Everyone   
3  215644   25M  50,000,000+  Free     0           Teen   
4     967  2.8M     100,000+  Free     0       Everyone   

                      Genres      Last Updated         Current Ver  \
0               Art & Design   January 7, 2018               1.0.0   
1  Art & Design;Pretend 

In [7]:
# Data preprocessing
# Convert 'Price', 'Reviews', 'Size', and 'Installs' to numeric (replace non-numeric values with 0)
playstore_data['Price'] = pd.to_numeric(playstore_data['Price'].str.replace('$', ''), errors='coerce').fillna(0)
playstore_data['Reviews'] = pd.to_numeric(playstore_data['Reviews'], errors='coerce').fillna(0)
playstore_data['Size'] = pd.to_numeric(playstore_data['Size'].str.replace('M', '').str.replace('k', '').str.replace(',', ''), errors='coerce').fillna(0)
playstore_data['Installs'] = pd.to_numeric(playstore_data['Installs'].str.replace('+', '').str.replace(',', ''), errors='coerce').fillna(0)


<ipython-input-7-d1fdef120fb5>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  playstore_data['Price'] = pd.to_numeric(playstore_data['Price'].str.replace('$', ''), errors='coerce').fillna(0)
<ipython-input-7-d1fdef120fb5>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  playstore_data['Installs'] = pd.to_numeric(playstore_data['Installs'].str.replace('+', '').str.replace(',', ''), errors='coerce').fillna(0)


In [8]:
# Encode 'Content Rating' column
playstore_data = pd.get_dummies(playstore_data, columns=['Content Rating'], drop_first=True)

In [9]:
# Select features based on the prediction goal
cols_to_use_playstore = ['Rating', 'Reviews', 'Size', 'Installs', 'Price']  # Include 'Rating' in features
playstore_data = playstore_data[cols_to_use_playstore].dropna()

In [10]:
# Separate features (X) and target (y)
X_playstore = playstore_data.drop('Rating', axis=1)
y_playstore = playstore_data['Rating']

In [13]:
# Split data into training and validation sets
X_train_playstore, X_valid_playstore, y_train_playstore, y_valid_playstore = train_test_split(X_playstore, y_playstore, test_size=0.2, random_state=42)

In [14]:
# Hyperparameter tuning using GridSearchCV
# Define model
model_grid = XGBRegressor()

In [15]:
# Define parameter grid
param_grid = {
    'n_estimators': [100, 150, 200, 250, 300]  # Adjust the range of values to try
}

In [16]:
# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model_grid, param_grid=param_grid, cv=5, scoring='neg_mean_absolute_error')


In [17]:
# Fit the model with training data
grid_search.fit(X_train_playstore, y_train_playstore)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'n_estimators': [100, 150, 200, 250, 300]},
             scoring='neg_mean_absolute_error')

In [18]:
# Get the best parameters
best_n_estimators = grid_search.best_params_['n_estimators']
print("Best n_estimators:", best_n_estimators)

Best n_estimators: 100


In [19]:
# Define a range of n_estimators values to test
n_estimators_list = [100, 150, 200, 250, 300]

In [20]:
# Initialize a list to store mean absolute errors
mae_results = []

# Loop through each n_estimators value
for n_estimators in n_estimators_list:
    # Initialize model with specific n_estimators
    model_grid = XGBRegressor(n_estimators=n_estimators)

    # Fit the model with training data
    model_grid.fit(X_train_playstore, y_train_playstore)

    # Predict using validation data
    predictions = model_grid.predict(X_valid_playstore)

    # Calculate Mean Absolute Error
    mae = mean_absolute_error(predictions, y_valid_playstore)

    # Display n_estimators and MAE
    print(f"n_estimators: {n_estimators}, Mean Absolute Error: {mae}")

    # Append the MAE to the list
    mae_results.append(mae)

n_estimators: 100, Mean Absolute Error: 0.3390541421183909
n_estimators: 150, Mean Absolute Error: 0.34213244039704477
n_estimators: 200, Mean Absolute Error: 0.3456280211628819
n_estimators: 250, Mean Absolute Error: 0.3501116018539559
n_estimators: 300, Mean Absolute Error: 0.3519573104292505
